In [ ]:
from utils import *
import getpass
from landsatxplore.earthexplorer import EarthExplorer
import tarfile

In [ ]:
bbox = read_kml_polygon("data/inputs/landsat/WA.kml")[1]

In [ ]:
user = getpass.getpass("Enter your username?")
password = getpass.getpass("Enter your password?")
ee = EarthExplorer(user, password)

In [ ]:
# query = get_landsat_search_query([67.45, -72.55, 67.55, -72.45])
query = get_landsat_search_query(bbox)
features = fetch_landsat_stac_server(query)
feat_dict = find_landsat_scenes_dict(features)

In [ ]:
required_keys = [k for k in list(feat_dict.keys()) if "12711" in k]
required_scenes = [feat_dict[k]["scene_id"] for k in required_keys]
required_scenes

In [ ]:
for scene in required_scenes:
    output_dir = "data/inputs/landsat"
    os.makedirs("data/inputs/landsat", exist_ok=True)
    print(f"Downloading {scene}")
    ee.download(scene, output_dir)

In [ ]:
tar_files = [f for f in glob.glob("data/inputs/landsat/**") if f.endswith(".tar")]
for tar_file in tar_files:
    extraction_dir = os.path.join(
        output_dir, os.path.splitext(os.path.basename(tar_file))[0]
    )
    os.makedirs(extraction_dir, exist_ok=True)
    with tarfile.open(tar_file) as tf:
        tf.extractall(extraction_dir)

In [ ]:
red = "data/inputs/landsat/LC08_L1GT_127111_20141103_20201016_02_T2/LC08_L1GT_127111_20141103_20201016_02_T2_B4.TIF"
green = "data/inputs/landsat/LC08_L1GT_127111_20141103_20201016_02_T2/LC08_L1GT_127111_20141103_20201016_02_T2_B3.TIF"
blue = "data/inputs/landsat/LC08_L1GT_127111_20141103_20201016_02_T2/LC08_L1GT_127111_20141103_20201016_02_T2_B2.TIF"

redf = flip_img(rasterio.open(red).read())[:, :, 0]
redf = apply_gamma(redf, 1.0, True)

greenf = flip_img(rasterio.open(green).read())[:, :, 0]
greenf = apply_gamma(greenf, 1.0, True)

bluef = flip_img(rasterio.open(blue).read())[:, :, 0]
bluef = apply_gamma(bluef, 1.0, True)


img = cv.merge([redf, bluef, greenf])
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
true_color_dir = "data/inputs/landsat/true_color"
os.makedirs(true_color_dir, exist_ok=True)
scenes_dirs = [
    dir for dir in glob.glob("data/inputs/landsat/**") if dir.endswith("_T2")
]
for dir in scenes_dirs:
    print(f"Now processing {os.path.basename(dir)}.")
    files = glob.glob(f"{dir}/**")
    r_band = list(filter(lambda f: f.endswith("_B4.TIF"), files))[0]
    g_band = list(filter(lambda f: f.endswith("_B3.TIF"), files))[0]
    b_band = list(filter(lambda f: f.endswith("_B2.TIF"), files))[0]
    true_bands = [r_band, g_band, b_band]
    make_landsat_true_color_scene(
        true_bands, f"{os.path.join(true_color_dir, os.path.basename(dir))}_TC.TIF"
    )

In [ ]:
true_color_ds_dir = "data/inputs/landsat/true_color_ds"
os.makedirs(true_color_ds_dir, exist_ok=True)
tc_files = glob.glob("data/inputs/landsat/true_color/**")
for tcf in tc_files:
    downsample_dataset(tcf, 0.2, os.path.join(true_color_ds_dir, os.path.basename(tcf)))

In [ ]:
show(
    rasterio.open(
        "data/inputs/landsat/true_color_ds/LC08_L1GT_127111_20150106_20201016_02_T2_TC.TIF"
    ).read()
)